
# LSMV Downstream MVP — SOP (End‑to‑End)

## 0) What this service does (TL;DR)
- Accepts an incoming report (JSON body or `.json` file upload).
- (For POC) “Masks” values (turns all values into `"1"`) so we can test flow without real PHI/PII.
- Simulates failures when `triggerFail=true` is provided.
- Logs every attempt to `submission.log`.
- Stores failed payloads in `failed.jsonl` for manual retries via `retry_runner.py`.



## 1) Project layout (expected)
```plaintext
lsmv-downstream-fastapi/
├─ app/
│  ├─ main.py             # FastAPI app; exposes /test-post and other utilities
│  ├─ poster.py           # HTTP post + logging + retry helpers
│  ├─ utils.py            # mask_json_values() and misc utilities
│  ├─ etl.py              # (ingest/clean helpers for tabular files)
│  ├─ json_generator.py   # (generate JSON records from DataFrame)
│  └─ parse_e2b_xml.py    # (XML → records)
├─ retry_runner.py        # manual retry CLI for failed.jsonl
├─ requirements.txt
├─ submission.log         # (generated)
└─ failed.jsonl           # (generated)
```



## 2) Environment setup (local)
1. **Create/activate a virtualenv**

```bash
python3 -m venv .venv
source .venv/bin/activate
```

2. **Install dependencies**
```bash
pip install -r requirements.txt
```

3. **Start the API (hot reload)**
```bash
uvicorn app.main:app --reload
```
Expected output:
```
Uvicorn running on http://127.0.0.1:8000
```



## 3) Endpoint 

### `POST /test-post`
**Input (either)**  
- JSON body (`Content-Type: application/json`)  
- File upload multipart with `file=@your.json`  

**Special testing flag:** `triggerFail: true` forces a 500‑style failure path.

**Flow:**
1. Reads payload (file or JSON).
2. If `triggerFail == true`, raises an error (simulated).
3. Otherwise masks values and returns masked JSON with 200.
4. Logs every attempt to `submission.log`.
5. On failure, also appends the original payload to `failed.jsonl`.



## 4) Smoke tests (curl)

Open a second terminal to run these while the server is on `http://localhost:8000`.


In [ ]:

# 4.1 Success (JSON body)
!curl -X POST http://localhost:8000/test-post \
  -H "Content-Type: application/json" \
  -d '{"reportId": "AEV-SUCCESS-101", "eventDate": "2025-08-15"}'


In [ ]:

# 4.2 Failure (simulate with triggerFail)
!curl -X POST http://localhost:8000/test-post \
  -H "Content-Type: application/json" \
  -d '{"reportId": "AEV-FAIL-101", "triggerFail": true}'


In [ ]:

# 4.3 File upload (optional)
!curl -X POST http://localhost:8000/test-post \
  -H "Content-Type: multipart/form-data" \
  -F "file=@/absolute/path/to/your.json"



## 5) Where to look: logs & failed queue

### 5.1 Submission log
```bash
cat submission.log
```

Example lines:
```
[2025-08-15T09:33:44.954984] FAILURE: AEV-FAIL-TEST → Simulated failure via triggerFail → Code 500
[2025-08-15T09:33:53.299559] SUCCESS: AEV-FAIL-TEST → Code 200
```

### 5.2 Failed queue (JSONL)
```bash
cat failed.jsonl
```
Example:
```json
{"reportId":"AEV-PERSIST-001","triggerFail":true,"retry_count":1}
```



## 6) Manual retry workflow

1. **Clean before test cycle (optional)**
```bash
: > submission.log
: > failed.jsonl
```

2. **Generate a failure**
```bash
curl -X POST http://localhost:8000/test-post   -H "Content-Type: application/json"   -d '{"reportId":"AEV-PERSIST-001","triggerFail":true}'
```

3. **Run retries**
```bash
python retry_runner.py --url http://localhost:8000/test-post --max 3
```
- `--url` is required.
- `--max` caps per‑payload retries.

4. **Check status**
```bash
cat failed.jsonl
tail -n 10 submission.log
```



## 7) Masking
`mask_json_values()` (in `app/utils.py`) replaces all values with `"1"` while keeping keys.  
Safe for testing without PHI.

## 8) Troubleshooting
- **500 but empty failed.jsonl** → Ensure `_save_failed_payload` is called in exception path.  
- **Port in use** → Kill the old process or change port.  
- **Import errors** → Run from repo root.

## 9) Render test
```bash
python retry_runner.py --url https://<your-render-host>/test-post --max 3
```

## 10) Paused for now
- Mule OAuth
- Postgres ledger
- Real outlet mapping

## 11) One‑screen demo
1. Start API
2. Send success
3. Send failure
4. Retry → show persistence
5. Edit/remove `triggerFail` → retry → success
